In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
raw = pd.read_csv('../ch28_powercomp/chapter28_powercomp.csv')

In [3]:
zone1 = raw[['DateTime', 'Temperature', 'Humidity', 'Wind Speed', 'general diffuse flows', 'diffuse flows', 'Zone 1 Power Consumption']]

In [4]:
zone1.columns = ['DateTime', 'Temperature', 'Humidity', 'Wind Speed', 'general diffuse flows', 'diffuse flows', 'zone1']

In [5]:
zone1.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,zone1
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620


In [6]:
# 스케일링
def zeroOneScaler(col):
    min_val = col.min()
    max_val = col.max()
    scaled_col = (col - min_val) / (max_val - min_val)
    return scaled_col

In [7]:
zone1['Temperature'] = (zone1['Temperature'] - np.min(zone1['Temperature'])) / (np.max(zone1['Temperature']) - np.min(zone1['Temperature']))
zone1['Humidity'] = (zone1['Humidity'] - np.min(zone1['Humidity'])) / (np.max(zone1['Humidity']) - np.min(zone1['Humidity']))
zone1['Wind Speed'] = (zone1['Wind Speed'] - np.min(zone1['Wind Speed'])) / (np.max(zone1['Wind Speed']) - np.min(zone1['Wind Speed']))
zone1['general diffuse flows'] = (zone1['general diffuse flows'] - np.min(zone1['general diffuse flows'])) / (np.max(zone1['general diffuse flows']) - np.min(zone1['general diffuse flows']))
zone1['diffuse flows'] = (zone1['diffuse flows'] - np.min(zone1['diffuse flows'])) / (np.max(zone1['diffuse flows']) - np.min(zone1['diffuse flows']))
zone1['zone1'] = (zone1['zone1'] - np.min(zone1['zone1'])) / (np.max(zone1['zone1']) - np.min(zone1['zone1']))

/var/folders/j4/ht7qhxvx4h53wf17rkc1typh0000gn/T/ipykernel_53578/1422999836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zone1['Temperature'] = (zone1['Temperature'] - np.min(zone1['Temperature'])) / (np.max(zone1['Temperature']) - np.min(zone1['Temperature']))
/var/folders/j4/ht7qhxvx4h53wf17rkc1typh0000gn/T/ipykernel_53578/1422999836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zone1['Humidity'] = (zone1['Humidity'] - np.min(zone1['Humidity'])) / (np.max(zone1['Humidity']) - np.min(zone1['Hu

In [8]:
def create_windows(data, window_size):
    X = []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
    return np.array(X)

In [21]:
w_size = 32

In [22]:
X = create_windows(zone1['zone1'], 16)
X = X.reshape(-1, w_size, 1)
splitIdx = (int)(len(X) * 0.8)
train_x = X[:splitIdx]
test_x = X[splitIdx:]

In [17]:
from LSTMAE import LSTMAE

In [25]:
window_size = 256
num_features = 1

model = LSTMAE(w_size, num_features)
model.build_model()
model.print_summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 32, 64)            16896     
                                                                 
 lstm_13 (LSTM)              (None, 32)                12416     
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 repeat_vector_3 (RepeatVec  (None, 32, 16)            0         
 tor)                                                            
                                                                 
 lstm_14 (LSTM)              (None, 32, 32)            6272      
                                                                 
 lstm_15 (LSTM)              (None, 32, 64)            24832     
                                                      

In [26]:
model.train(train_x, set_epochs = 50)
model.plot_training_history()

Epoch 1/50


2025-07-03 20:08:27.041958: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-07-03 20:08:27.631125: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-07-03 20:08:27.692978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-07-03 20:08:27.752760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-07-03 20:08:27.873235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-07-03 20:08:27.987923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-07-03 20:08:28.110272: I tensorflow/core/grappler/optimizers/cust

  6/590 [..............................] - ETA: 9:21 - loss: 0.3994 - mae: 0.3994

KeyboardInterrupt: 